# Day 14: Extended Polymerization

In [1]:
from collections import Counter

In [2]:
def load_input(filename):
    rules = {}
    with open(filename) as fr:
        root = fr.readline().strip()
        fr.readline()
        for line in fr.readlines():
            f, t = line.strip().split(' -> ')
            rules[f] = t
    return root, rules
        

In [3]:
root, rules = load_input('14-sample.txt')
assert root == 'NNCB'
assert len(rules) == 16
assert rules['HB'] == 'C'

In [4]:
def polymerize(seq, rules):
    seq = iter(seq)
    a = next(seq)
    yield a
    for b in seq:
        yield rules[''.join([a, b])]
        yield b
        a = b

In [5]:
root, rules = load_input('14-sample.txt')
assert ''.join(polymerize(root, rules)) == 'NCNBCHB'

In [6]:
initial, rules = load_input('14-sample.txt')
for _ in range(4):
    initial = ''.join(polymerize(initial, rules))
assert initial  == 'NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB'

This polymer grows quickly. After step $5$, it has length $97$; After step $10$, it has length $3073$. After step $10$, `B` occurs $1749$ times, `C` occurs $298$ times, `H` occurs $161$ times, and `N` occurs $865$ times; taking the quantity of the most common element `(B, 1749)` and subtracting the quantity of the least common element `(H, 161)` produces $1749 - 161 = 1588$.



In [7]:
def get_polymer(initial, rules, level):
    result = initial
    for _ in range(level):
        result = ''.join(polymerize(result, rules))
    return result

In [8]:
initial, rules = load_input('14-sample.txt')
assert len(get_polymer(initial, rules, 5)) == 97

polymer = get_polymer(initial, rules, 10)
assert len(polymer) == 3073
assert polymer.count('B') == 1749
assert polymer.count('C') == 298
assert polymer.count('H') == 161
assert polymer.count('N') == 865

In [9]:
def solution_part_one(filename, iterations=10):
    initial, rules = load_input(filename)
    polymer = get_polymer(initial, rules, iterations)
    c = Counter(polymer)
    most_common = c.most_common()
    _, max_count = most_common[0]
    _, min_count = most_common[-1]
    return max_count - min_count
    

In [10]:
assert solution_part_one('14-sample.txt') == 1588

## Solution part one

In [11]:
sol = solution_part_one('14-input.txt')
print(f"Solution part one: {sol}")

Solution part one: 3906


## Part two

In [12]:
import itertools
import time
import datetime
import functools

def pairs(seq):
    a, b = itertools.tee(seq, 2)
    next(b)
    for i0, i1 in zip(a, b):
        yield ''.join([i0, i1])


assert list(pairs('ABC')) == ['AB', 'BC']
assert list(pairs('NNCB')) == ['NN', 'NC', 'CB']

In [13]:
initial, rules = load_input('14-simple.txt')
print(get_polymer(initial, rules, 10))

ABCABCBBCAACABCABCBBCAACBBCABCBBCAACABCAACBBCAACABCABCBBCAACABCABCBBCAACBBCABCBBCAACABCAACBBCAACBBCABCBBCAACABCABCBBCAACBBCABCBBCAACABCAACBBCAACABCABCBBCAACABCAACBBCAACBBCABCBBCAACABCAACBBCAACABCABCBBCAACABCABCBBCAACBBCABCBBCAACABCAACBBCAACABCABCBBCAACABCABCBBCAACBBCABCBBCAACABCAACBBCAACBBCABCBBCAACABCABCBBCAACBBCABCBBCAACABCAACBBCAACABCABCBBCAACABCAACBBCAACBBCABCBBCAACABCAACBBCAACBBCABCBBCAACABCABCBBCAACBBCABCBBCAACABCAACBBCAACABCABCBBCAACABCABCBBCAACBBCABCBBCAACABCAACBBCAACBBCABCBBCAACABCABCBBCAACBBCABCBBCAACABCAACBBCAACABCABCBBCAACABCAACBBCAACBBCABCBBCAACABCAACBBCAACABCABCBBCAACABCABCBBCAACBBCABCBBCAACABCAACBBCAACABCABCBBCAACABCAACBBCAACBBCABCBBCAACABCAACBBCAACBBCABCBBCAACABCABCBBCAACBBCABCBBCAACABCAACBBCAACABCABCBBCAACABCAACBBCAACBBCABCBBCAACABCAACBBCAACABCABCBBCAACABCABCBBCAACBBCABCBBCAACABCAACBBCAACABCABCBBCAACABCABCBBCAACBBCABCBBCAACABCAACBBCAACBBCABCBBCAACABCABCBBCAACBBCABCBBCAACABCAACBBCAACABCABCBBCAACABCAACBBCAACBBCABCBBCAACABCAACBBCAACABCABCBBCAACABCABCBBCAACBBCABCBBCAACABCA

In [14]:
def drop_zeros(dd):
    return Counter({ k: dd[k] for k in dd if dd[k] != 0})
             
counter = Counter('abcx')
counter['x'] -= 1
counter = drop_zeros(counter)
assert 'x' not in counter
assert len(counter) == 3

In [15]:
def matraca(filename, level, tron=False, show_cache_stats=False):
    initial, rules = load_input(filename)
    result = Counter()
    num_symbols = len(set(rules))
    max_level = level
    
    @functools.lru_cache(40*num_symbols*num_symbols)
    def sumsub(root, level, tron=False):
        delta = max_level - level
        if tron: 
            print(" " * level, f"- Called sumsub({root!r}, level={level!r}) [delta: {delta}")
        if delta < 1:
            return {}
        elif delta == 1:
            left, right = root
            ax = rules[root]
            return Counter([left, ax, right])
        if delta > 1:
            left, right = root  
            ax = rules[root]
            result = Counter({ax: -1})
            left_branch = f'{left}{ax}'
            result.update(sumsub(left_branch, level+1, tron))
            right_branch = f'{ax}{right}'
            result.update(sumsub(right_branch, level+1, tron))
            return result
    sumsub.cache_clear()

    if level == 0:
        return Counter(initial)
    if tron:
        polimer = get_polymer(initial, rules, max_level)
        print(f'polimer por level {level}: {polimer}')
        print(f'root por level {level}: {initial[0:2]}')
    result = Counter()
    for a, b in pairs(initial):
        subpart = Counter()
        if tron:
            print(f"Analyzing {a}{b}")
        subpart.update(sumsub(f"{a}{b}", 0))
        if tron: 
            polymer = get_polymer(f"{a}{b}", rules, max_level)
            print(f" - polymer: {polymer}")
            print(f" - subtree {a}{b} delivers: {subpart}")
        subpart[b] -= 1
        result.update(subpart)
    if b in result:
        result[b] += 1
    if show_cache_stats:
        print(sumsub.cache_info())
    return drop_zeros(result)

print(matraca('14-simple.txt', 3))

Counter({'C': 3, 'B': 3, 'A': 3})


In [16]:
assert matraca('14-simple.txt', 0) == Counter({'A': 1, 'B': 1})
assert matraca('14-simple.txt', 1) == Counter({'A': 1, 'B': 1, 'C': 1})
assert matraca('14-simple.txt', 2) == Counter({'A': 2, 'B': 2, 'C': 1})
assert matraca('14-simple.txt', 3) == Counter({'A': 3, 'B': 3, 'C': 3})

In [17]:
initial, rules = load_input('14-simple.txt')
for level in range(4, 12):
    polymer = get_polymer(initial, rules, level)
    assert matraca('14-simple.txt', level) == Counter(polymer)

### Tests with sample

In [18]:
assert matraca('14-sample.txt', 0) == Counter('NNCB')
assert matraca('14-sample.txt', 1) == Counter('NCNBCHB')
assert matraca('14-sample.txt', 2) == Counter('NBCCNBBBCBHCB')
assert matraca('14-sample.txt', 3) == Counter('NBBBCNCCNBBNBNBBCHBHHBCHB')

In [19]:
initial, rules = load_input('14-sample.txt')
for level in range(4, 22):
    polymer = get_polymer(initial, rules, level)
    assert matraca('14-sample.txt', level) == Counter(polymer)
    print(level, True)

4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True
19 True
20 True
21 True


In [20]:
def solution_part_two(filename, iterations=10):
    initial, rules = load_input(filename)
    c = matraca(filename, iterations)
    most_common = c.most_common()
    _, max_count = most_common[0]
    _, min_count = most_common[-1]
    return max_count - min_count

In [21]:
assert solution_part_two('14-sample.txt', 10) == 1588

In [22]:
assert solution_part_two('14-sample.txt', 40) == 2188189693529

## Solution part two

In [23]:
sol = solution_part_two('14-input.txt', 40)
print(f"Solution part two: {sol}")

Solution part two: 4441317262452
